# Color Wheel

For this example we'll be making a simple color picker that's powered by python.

In [1]:
import sys
import math
import purly

To set things up we'll need to stand up our model server. By default the server has a max refresh rate of `25` hertz. However for this example to be convincingly smooth we'll want to bump that up to about `60`. If you want to unlock the refresh rate set `refresh=None`. We'll then hook up a layout object to a model resource (see the introductory example  or [read the docs](https://github.com/rmorshea/purly#purly) if this doesn't make sense).

In [2]:
from example_utils import localhost

# increase model server refresh cap to 60 hertz.
purly.state.Machine(refresh=60).daemon()

# name the layout resource "color-wheel" and connect to the update stream
websocket_url = localhost('ws', 8000) + '/model/color-wheel/stream'
layout = purly.Layout(websocket_url)

[2018-07-26 23:57:10 -0700] [9255] [INFO] Goin' Fast @ http://127.0.0.1:8000
[2018-07-26 23:57:10 -0700] [9255] [INFO] Starting worker [9255]
[2018-07-26 23:57:13 -0700] [9255] [ERROR] Traceback (most recent call last):
  File "/home/ryan/conda/envs/Py3.6/lib/python3.6/site-packages/sanic/app.py", line 556, in handle_request
    response = await response
  File "/home/ryan/Desktop/purly/purly/state.py", line 15, in _index
    return await response.file(absolute)
  File "/home/ryan/conda/envs/Py3.6/lib/python3.6/site-packages/sanic/response.py", line 304, in file
    async with open_async(location, mode='rb') as _file:
  File "/home/ryan/conda/envs/Py3.6/lib/python3.6/site-packages/aiofiles/base.py", line 83, in __aenter__
    self._obj = yield from self._coro
  File "/home/ryan/conda/envs/Py3.6/lib/python3.6/site-packages/aiofiles/threadpool/__init__.py", line 36, in _open
    f = yield from loop.run_in_executor(executor, cb)
  File "/home/ryan/conda/envs/Py3.6/lib/python3.6/concurrent

To do this we'll need a simple function that will create an HSL (hue, saturation, and lightness) color string. The function will accept the radius of the color picker wheel, and the x-y position of the mouse in order to select a color based on the angle around the circle, and distance from the center.

In [3]:
def hsl(radius, x, y):
    """Return an HSL color string."""
    x -= radius
    y -= radius
    unit_radius = int((x ** 2 + y **2) ** 0.5) / radius
    degrees = int(math.atan2(x, y) * 180 / math.pi)
    return "hsl(%s, 100%%, %s%%)" % (degrees, unit_radius * 100)

Next we'll need style up the the color wheel, and a selection indicator.

In [4]:
radius = 50
wheel = layout.html('div')
wheel.style.update(
    height="%spx" % (radius * 2),
    width="%spx" % (radius * 2),
    backgroundColor="hsl(120, 100%, 50%)",
    borderRadius="50%",
)

In [5]:
selection = layout.html('div')
selection.style.update(
    height="20px",
    width="20px",
    backgroundColor="hsl(120, 100%, 50%)",
)

In [6]:
layout.children.append(wheel)
layout.children.append(selection)
layout.sync()
layout

You should now see the color wheel and selector indicator in the output above!

However when you mouse over the wheel nothing happens, so we'll need to hook into some mouse events to make the display animate. To do that requires the `onMouseMove` and `onClick` events which can be captures via the `on` decorator of the `wheel` element. The underlying logic of each is actually pretty simple:

1. `onMouseMove`: if the mouse moves over the color wheel, then set the color wheel to the HSL color that corresponds to its x-y position.
2. `onClick`: if the mouse clicks on the wheel, set the selection indicator to the corresponding HSL color.

In [ ]:
@wheel.on('MouseMove')
def cause_color_change(clientX, clientY):
    wheel.style["backgroundColor"] = hsl(50, clientX, clientY)

@wheel.on("Click")
def cause_color_select(clientX, clientY):
    selection.style["backgroundColor"] = hsl(50, clientX, clientY)

Finally we'll need to serve up our event handlers in order to animate.

In [ ]:
layout.serve()

Now mouse over the wheel and try selecting a color!